# Setting

- We have k chunks to estimate
- Each chunk i already has $m_i$ past results $R_{ij} \sim Lap(1/n_i \epsilon_{ij})$
- Linear combination: $Q_{ij} := \frac{n_i}{n} \gamma_{ij}(q_i + R_{ij})$
- We need $\sum_j \gamma_{ij} = 1$ for each chunk $i$ to have an unbiased estimate
- Goal: output an estimate $Q = Q_{10} + \dots + Q_{1m_1} +\dots + Q_{k0} + \dots + Q_{km_k}$
- Accuracy constraint: $\Pr[|Q-q|>\alpha] < \beta$
- Simplifying budget constraint: $R_{i0} \sim Lap(1/n_\epsilon)$ (same budget for everyone!). That's a weird constraint if we have chunks that are already good, let's see if we can set some epsilons to infinity. In the general case we could try an ILP or even some gradient descent on the budget vector?

In [23]:
import numpy as np

from precycle.utils.utility_theorems import get_epsilon_isotropic_laplace_concentration, binary_search

In [24]:
def minimum_variance(epsilons, noises, chunk_sizes):
    n_chunks = len(epsilons)
    chunk_noises = np.zeros(n_chunks)
    for chunk_id in range(n_chunks):
        # The variance has an extra 2/n_i^2 factor but it doesn't matter at the chunk level
        laplace_coefficients = epsilons[chunk_id] ** 2
        # See optimal variance reduction and lemma in Overleaf
        laplace_coefficients = laplace_coefficients / sum(laplace_coefficients)
        chunk_noises[chunk_id] = np.dot(laplace_coefficients, noises[chunk_id])
        
    chunk_coefficients = chunk_sizes / sum(chunk_sizes)
    aggregated_noise_total = np.dot(chunk_coefficients, chunk_noises)
    return aggregated_noise_total

In [25]:
# TODO: given epsilon and past results, find beta

def monte_carlo_beta(existing_epsilons, chunk_sizes, fresh_epsilon, alpha, N=1_000_000):
    
    # Add fresh epsilon
    epsilons = [
        np.append(eps_by_chunk, fresh_epsilon) for eps_by_chunk in existing_epsilons
    ]
    
    # TODO: heuristic to drop some chunks?
    
    # Vectorized code with a batch dimension corresponding to N
    n_chunks = len(epsilons)
    n = sum(chunk_sizes)
    chunk_noises = np.zeros((N, n_chunks))
    for chunk_id in range(n_chunks):
        # The final laplace scale (Q_ij), already scaled by n_i/n * eps^2/sum(eps^2)
        single_chunk_laplace_scale = epsilons[chunk_id] / (n * np.sum(epsilons[chunk_id] ** 2))
        laplace_scale = np.repeat([single_chunk_laplace_scale], N, axis=0)
        laplace_noises = np.random.laplace(scale=laplace_scale)
        
        
        # Optimal average for that chunk, N times
        chunk_noises[:, chunk_id] = np.sum(laplace_noises, axis=1)
        
    aggregated_noise_total = np.sum(chunk_noises, axis=1)
    beta = np.sum(aggregated_noise_total > alpha) / N
    return beta


In [26]:
existing_epsilons = [
    np.array([1, 0.5, 2]),
    np.array([0.3, 2.1])
]

chunk_sizes = [100, 200]
fresh_epsilon = 0.1
alpha = 0.001

monte_carlo_beta(existing_epsilons, chunk_sizes, fresh_epsilon, alpha, N=10_000_000)

0.3509521

In [27]:
# TODO: binary search for epsilon

def binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha, beta, N=1_000_000):
    get_beta_fn = lambda eps: monte_carlo_beta(existing_epsilons=existing_epsilons, chunk_sizes=chunk_sizes, fresh_epsilon=eps, alpha=alpha, N=N)
    
    # Worst case: ignore all existing epsilons, use loose concentration bound over the k chunks
    epsilon_high = get_epsilon_isotropic_laplace_concentration(a=alpha, b=beta, n=sum(chunk_sizes), k=len(chunk_sizes))
    
    return binary_search(get_beta_fn=get_beta_fn, beta=beta, epsilon_high=epsilon_high)

In [38]:
binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=0.01, N=100_000)

0 < 24.976507591763784 < 49.95301518352757 gives beta=0.0016. Target 0.01
0 < 12.488253795881892 < 24.976507591763784 gives beta=0.03311. Target 0.01
12.488253795881892 < 18.73238069382284 < 24.976507591763784 gives beta=0.00636. Target 0.01
12.488253795881892 < 15.610317244852366 < 18.73238069382284 gives beta=0.01489. Target 0.01
15.610317244852366 < 17.171348969337604 < 18.73238069382284 gives beta=0.00933. Target 0.01
15.610317244852366 < 16.390833107094984 < 17.171348969337604 gives beta=0.01172. Target 0.01
16.390833107094984 < 16.781091038216296 < 17.171348969337604 gives beta=0.01069. Target 0.01
16.781091038216296 < 16.97622000377695 < 17.171348969337604 gives beta=0.01062. Target 0.01
16.97622000377695 < 17.07378448655728 < 17.171348969337604 gives beta=0.00998. Target 0.01
16.97622000377695 < 17.025002245167116 < 17.07378448655728 gives beta=0.0101. Target 0.01
17.025002245167116 < 17.049393365862198 < 17.07378448655728 gives beta=0.01055. Target 0.01
17.049393365862198 < 17

17.070156253114007

In [37]:
binary_search_monte_carlo(existing_epsilons, chunk_sizes=[10,290], alpha=alpha, beta=0.01, N=1_000_000)

0 < 24.976507591763784 < 49.95301518352757 gives beta=0.001284. Target 0.01
0 < 12.488253795881892 < 24.976507591763784 gives beta=0.03216. Target 0.01
12.488253795881892 < 18.73238069382284 < 24.976507591763784 gives beta=0.006576. Target 0.01
12.488253795881892 < 15.610317244852366 < 18.73238069382284 gives beta=0.014572. Target 0.01
15.610317244852366 < 17.171348969337604 < 18.73238069382284 gives beta=0.009919. Target 0.01
15.610317244852366 < 16.390833107094984 < 17.171348969337604 gives beta=0.011945. Target 0.01
16.390833107094984 < 16.781091038216296 < 17.171348969337604 gives beta=0.010878. Target 0.01
16.781091038216296 < 16.97622000377695 < 17.171348969337604 gives beta=0.01043. Target 0.01
16.97622000377695 < 17.07378448655728 < 17.171348969337604 gives beta=0.010036. Target 0.01
17.07378448655728 < 17.122566727947444 < 17.171348969337604 gives beta=0.009975. Target 0.01
17.07378448655728 < 17.09817560725236 < 17.122566727947444 gives beta=0.01012. Target 0.01
17.0981756072

17.1073697700528

In [ ]:
# TODO: Why do we get slightly different epsilons? What are our exact guarantees?

## Aggregation with no VR

Let's start with chunks of same size and no past epsilons

In [8]:
import plotly.express as px
import pandas as pd

In [9]:

total_size = 100_000

d = []

for beta in [1e-4, 1e-3]:
    for alpha in [0.001, 0.01]:
        for k in [1,2,3,4,5,6,7,10,12,14,16,18,20]:
            existing_epsilons = [np.array([])]*k
            chunk_sizes = [100_000 // k] * k
            for bound in ["monte_carlo", "concentration"]:
                if bound == "monte_carlo":
                    try:
                        epsilon = binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)
                    except AssertionError as e:
                        print(e)
                        epsilon = np.NAN
                else:
                    epsilon = get_epsilon_isotropic_laplace_concentration(a=alpha, b=beta, n=total_size, k=k)
                d.append(dict(
                    alpha=alpha,
                    beta=beta,
                    k=k,
                    epsilon=epsilon,
                    bound=bound        
                ))
df = pd.DataFrame(d)

KeyboardInterrupt: 

In [ ]:
df

,alpha,beta,k,epsilon,bound
0,0.001,0.0001,1,0.086257,monte_carlo
1,0.001,0.0001,1,0.092103,concentration
2,0.001,0.0001,2,0.103128,monte_carlo
3,0.001,0.0001,2,0.280113,concentration
4,0.001,0.0001,3,0.118174,monte_carlo
...,...,...,...,...,...
99,0.010,0.0010,16,0.031192,concentration
100,0.010,0.0010,18,0.019340,monte_carlo
101,0.010,0.0010,18,0.033084,concentration
102,0.010,0.0010,20,0.020297,monte_carlo


In [ ]:
px.line(df, x='k', y='epsilon', color='bound', facet_col="alpha", facet_row="beta", title=f'epsilon vs k')

## VR on a single chunk

In [18]:
%load_ext autoreload
%autoreload 2

In [ ]:
#TODO: check if epsilon=0 can work first

In [20]:

existing_epsilons = [np.array([0.1])]
chunk_sizes = [100_000]
alpha = 0.01
beta = 0.001
binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=100)

0 < 0.0034538776394910683 < 0.006907755278982137 gives beta=0.0. Target 0.001
0 < 0.0017269388197455342 < 0.0034538776394910683 gives beta=0.0. Target 0.001
0 < 0.0008634694098727671 < 0.0017269388197455342 gives beta=0.0. Target 0.001
0 < 0.00043173470493638354 < 0.0008634694098727671 gives beta=0.0. Target 0.001
0 < 0.00021586735246819177 < 0.00043173470493638354 gives beta=0.0. Target 0.001
0 < 0.00010793367623409589 < 0.00021586735246819177 gives beta=0.0. Target 0.001
0 < 5.396683811704794e-05 < 0.00010793367623409589 gives beta=0.0. Target 0.001
0 < 2.698341905852397e-05 < 5.396683811704794e-05 gives beta=0.0. Target 0.001
0 < 1.3491709529261986e-05 < 2.698341905852397e-05 gives beta=0.0. Target 0.001
0 < 6.745854764630993e-06 < 1.3491709529261986e-05 gives beta=0.0. Target 0.001
0 < 3.3729273823154964e-06 < 6.745854764630993e-06 gives beta=0.0. Target 0.001
0 < 1.6864636911577482e-06 < 3.3729273823154964e-06 gives beta=0.0. Target 0.001
0 < 8.432318455788741e-07 < 1.686463691157

KeyboardInterrupt: 

In [11]:
for beta in [1e-4, 1e-3]:
    for alpha in [0.001, 0.01]:
        for existing_eps in [0.001, 0.1, 0.5, 1, 10]:
            existing_epsilons = [np.array([existing_eps])]
            chunk_sizes = [100_000]
            for bound in ["monte_carlo"]:
                if bound == "monte_carlo":
                    try:
                        epsilon = binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)
                    except AssertionError as e:
                        print(e)
                        epsilon = np.NAN
                else:
                    epsilon = get_epsilon_isotropic_laplace_concentration(a=alpha, b=beta, n=total_size, k=k)
                d.append(dict(
                    alpha=alpha,
                    beta=beta,
                    existing_eps=existing_eps,
                    epsilon=epsilon,
                    bound=bound        
                ))
df = pd.DataFrame(d)

KeyboardInterrupt: 

In [ ]:
px.line(df, x='existing_eps', y='epsilon', color='bound', facet_col="alpha", facet_row="beta", title=f'epsilon vs k')